# Trabalho FINAL — Classificação de Revisões de Filmes
- Equipe: Pioneiros de Dados
  - clebr
  - vi...

## Instalação das Bibliotecas

In [2]:
!pip install catboost
import pandas as pd
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes     import MultinomialNB
from sklearn.metrics         import accuracy_score ,balanced_accuracy_score
from sklearn.ensemble        import RandomForestClassifier

import nltk
from nltk.corpus import stopwords

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling  import RandomOverSampler

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.5 MB/s eta 0:00:00


In [3]:
#!pip install catboost
from sklearn.linear_model import LogisticRegression, RidgeClassifier, SGDClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier, VotingClassifier,AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

import pandas as pd
from sklearn.metrics import balanced_accuracy_score
from sklearn.utils.class_weight       import compute_class_weight


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


#Acesso ao drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
from imblearn.over_sampling import SMOTE
from collections import Counter

from nltk.stem import PorterStemmer
import spacy
import numpy as np

## Carregar e preparar os dados

In [32]:
train_critic = pd.read_csv("file:///content/drive/MyDrive/Colab Notebooks/INF_0619/train_critic.csv", lineterminator='\n')
test_critic  = pd.read_csv("file:///content/drive/MyDrive/Colab Notebooks/INF_0619/test_critic.csv" , lineterminator='\n')

Preparando dados - retirando review nulos

In [33]:
train_critic = train_critic[train_critic['Review'].notna() & train_critic['Review'].str.strip().astype(bool)]
test_critic  = test_critic [test_critic ['Review'].notna() & test_critic ['Review'].str.strip().astype(bool)]

In [34]:
train_critic['Review'] = train_critic['Review'].apply(lambda x: x.lower())
test_critic ['Review'] = test_critic ['Review'].apply(lambda x: x.lower())

stemmer = PorterStemmer()
nlp = spacy.load('en_core_web_sm')

# Funçoes
def apply_stemming(text):
    tokens = word_tokenize(text)
    stemmed = [stemmer.stem(word) for word in tokens]
    return ' '.join(stemmed)
def apply_lemmatization(text):
    doc = nlp(text)
    lemmatized = [token.lemma_ for token in doc]
    return ' '.join(lemmatized)

# Aplicar as transformações na coluna Review
train_critic['Review'] = train_critic['Review'].dropna().apply(apply_lemmatization)
test_critic ['Review'] = test_critic ['Review'].dropna().apply(apply_lemmatization)

# Amostragem

In [35]:
data = train_critic

X = data[['Show', 'Review']]
y = data['Sentiment']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [36]:
##########################################################################################################################################

# SMOTE

# Convert text data to numerical representation using TF-IDF
vectorizer = TfidfVectorizer(stop_words='english', max_features=100, max_df=0.85, min_df=10, ngram_range=(1, 3))

X_train_vec   = vectorizer.fit_transform(X_train['Review'])
X_valid_smote = vectorizer.    transform(X_valid['Review'])

smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_vec, y_train)

# volumetrias
print('\nPré balanceamento')
print(Counter(y))
print('\nPós balanceamento')
print('Treino:    ',Counter(y_train_smote))
print('Validação: ',Counter(y_valid      ))


Pré balanceamento
Counter({1: 9995, 0: 1836})

Pós balanceamento
Treino:     Counter({1: 7992, 0: 7992})
Validação:  Counter({1: 2003, 0: 364})


In [37]:
##########################################################################################################################################

# SIMPLE UNDERSAMPLE

# Converter X para índices porque Random*Sampler trabalha com arrays/matrizes
X_indices = X_train.index.values.reshape(-1, 1)

# Inicializar sampler
sampler = RandomUnderSampler(random_state=42)

# Aplicar o *sample
_, y_resampled_indices = sampler.fit_resample(X_indices, y_train)

# Filtrar o DataFrame original com os índices resampleados
balanced_data = data.iloc[y_resampled_indices.to_numpy().flatten().astype(float)]

# Divide train e test usando somente o conjunto train_critic
X_train_under = balanced_data[['Show', 'Review']]
y_train_under = balanced_data['Sentiment']


# volumetrias
print('\nPré balanceamento')
print(Counter(data['Sentiment']))
print('\nPós balanceamento')
print('Treino:    ',Counter(y_train_under))
print('Validação: ',Counter(y_valid      ))


Pré balanceamento
Counter({1: 9995, 0: 1836})

Pós balanceamento
Treino:     Counter({1: 1472, 0: 1472})
Validação:  Counter({1: 2003, 0: 364})


In [38]:
##########################################################################################################################################

# SIMPLE UNDERSAMPLE

# Converter X para índices porque Random*Sampler trabalha com arrays/matrizes
X_indices = X_train.index.values.reshape(-1, 1)

# Inicializar sampler
sampler = RandomOverSampler(random_state=42)

# Aplicar o *sample
_, y_resampled_indices = sampler.fit_resample(X_indices, y_train)

# Filtrar o DataFrame original com os índices resampleados
balanced_data = data.iloc[y_resampled_indices.to_numpy().flatten().astype(float)]

# Divide train e test usando somente o conjunto train_critic
X_train_over = balanced_data[['Show', 'Review']]
y_train_over = balanced_data['Sentiment']

# volumetrias
print('\nPré balanceamento')
print(Counter(data['Sentiment']))
print('\nPós balanceamento')
print('Treino:    ',Counter(y_train_over))
print('Validação: ',Counter(y_valid      ))


Pré balanceamento
Counter({1: 9995, 0: 1836})

Pós balanceamento
Treino:     Counter({0: 7992, 1: 7992})
Validação:  Counter({1: 2003, 0: 364})


In [39]:
##########################################################################################################################################

# PESOS POPULACIONAIS

classes = y_train.to_frame()['Sentiment'].unique()

weights = compute_class_weight(
    class_weight='balanced',
    classes=np.array(classes),
    y=y_train.to_frame()['Sentiment']
)

class_weights = dict(zip(classes, weights))
class_weights

{1: 0.5920920920920921, 0: 3.214673913043478}

# Vetorizando

In [40]:
for i in ['_under','_over','']:

  vectorizer = TfidfVectorizer(stop_words='english', max_features=100, max_df=0.85, min_df=10, ngram_range=(1, 3))

  globals()['X_train%s'%i] = vectorizer.fit_transform(globals()['X_train%s'%i]['Review'])
  globals()['X_valid%s'%i] = vectorizer.    transform(X_valid['Review'])

# Modelos

In [41]:
# Dicionário com modelos
models = {
    'Random Forest': RandomForestClassifier(random_state=42, n_estimators=10, max_features=2),
    'SVC(gamma=2, C=1)': SVC(gamma=2, C=1),
    'SVC(kernel="linear",C=0.025,probability=True)': SVC(kernel="linear",C=0.025,probability=True),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Radius Neighbors': RadiusNeighborsClassifier(radius=10),
    'Gradient Boosting': GradientBoostingClassifier(),
    'XGBoost': XGBClassifier(),
    'CatBoost': CatBoostClassifier(verbose=0),
    'Neural Network': MLPClassifier(alpha=1),
    'Stochastic Gradient Descent': SGDClassifier(),
    'Ridge Classifier': RidgeClassifier(),
    'AdaBoostClassifier': AdaBoostClassifier(),
    'MultinomialNB': MultinomialNB()
    }


# Dicionário com modelos
models_com_peso = {
    'Random Forest': RandomForestClassifier(random_state=42, n_estimators=10, max_features=2, class_weight = class_weights),
    'SVC(gamma=2, C=1)': SVC(gamma=2, C=1, class_weight = class_weights),
    'SVC(kernel="linear",C=0.025,probability=True)': SVC(kernel="linear",C=0.025,probability=True, class_weight = class_weights),
    'XGBoost': XGBClassifier(class_weight = class_weights),
    'Stochastic Gradient Descent': SGDClassifier(class_weight = class_weights),
    'Ridge Classifier': RidgeClassifier(class_weight = class_weights),
    }

# Treinar

Smote

In [46]:
results = []

# .astype(str)

for name, model in models.items():
  try:
    model.fit(X_train_smote, y_train_smote)
    results.append({  'Model'    : name
                    , 'Treino'   : balanced_accuracy_score(y_train_smote, model.predict(X_train_smote))
                    , 'Validação': balanced_accuracy_score(y_valid, model.predict(X_valid_smote))})
  except:
    None
# Exibir resultados em um DataFrame
results_df_smote = pd.DataFrame(results)
print(results_df_smote.sort_values(by='Validação', ascending=False))

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


                                            Model    Treino  Validação
12                                  MultinomialNB  0.650713   0.613733
8                                  Neural Network  0.656031   0.608982
10                               Ridge Classifier  0.662600   0.607360
2   SVC(kernel="linear",C=0.025,probability=True)  0.652402   0.602743
9                     Stochastic Gradient Descent  0.668731   0.600743
11                             AdaBoostClassifier  0.681494   0.598383
5                               Gradient Boosting  0.741929   0.597491
0                                   Random Forest  0.917167   0.556878
3                             K-Nearest Neighbors  0.770521   0.555695
7                                        CatBoost  0.837963   0.554377
1                               SVC(gamma=2, C=1)  0.893956   0.539013
6                                         XGBoost  0.822885   0.525878
4                                Radius Neighbors  0.500000   0.500000


Simple undersample

In [47]:
results = []

for name, model in models.items():
  try:
    model.fit(X_train_under, y_train_under)
    results.append({  'Model'    : name
                    , 'Treino'   : balanced_accuracy_score(y_train_under, model.predict(X_train_under))
                    , 'Validação': balanced_accuracy_score(y_valid, model.predict(X_valid_under))})
  except:
    None

# Exibir resultados em um DataFrame
results_df_under = pd.DataFrame(results)
print(results_df_under.sort_values(by='Validação', ascending=False))

                                            Model  Treino  Validação
12                                  MultinomialNB     1.0   0.503078
0                                   Random Forest     1.0   0.500626
11                             AdaBoostClassifier     1.0   0.500250
4                                Radius Neighbors     0.5   0.500000
6                                         XGBoost     1.0   0.500000
7                                        CatBoost     1.0   0.500000
5                               Gradient Boosting     1.0   0.499501
1                               SVC(gamma=2, C=1)     1.0   0.498214
3                             K-Nearest Neighbors     1.0   0.498214
2   SVC(kernel="linear",C=0.025,probability=True)     1.0   0.488456
8                                  Neural Network     1.0   0.488456
9                     Stochastic Gradient Descent     1.0   0.488456
10                               Ridge Classifier     1.0   0.488456


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Simple oversample

In [48]:
results = []

for name, model in models.items():
  try:
    model.fit(X_train_over, y_train_over)
    results.append({  'Model'    : name
                    , 'Treino'   : balanced_accuracy_score(y_train_over, model.predict(X_train_over))
                    , 'Validação': balanced_accuracy_score(y_valid, model.predict(X_valid_over))})
  except:
    None

# Exibir resultados em um DataFrame
results_df_over = pd.DataFrame(results)
print(results_df_over.sort_values(by='Validação', ascending=False))

                                            Model  Treino  Validação
11                             AdaBoostClassifier     1.0   0.504489
12                                  MultinomialNB     1.0   0.503078
0                                   Random Forest     1.0   0.500626
5                               Gradient Boosting     1.0   0.500430
4                                Radius Neighbors     0.5   0.500000
6                                         XGBoost     1.0   0.500000
7                                        CatBoost     1.0   0.500000
8                                  Neural Network     1.0   0.498214
9                     Stochastic Gradient Descent     1.0   0.498214
1                               SVC(gamma=2, C=1)     1.0   0.488456
2   SVC(kernel="linear",C=0.025,probability=True)     1.0   0.488456
3                             K-Nearest Neighbors     1.0   0.488456
10                               Ridge Classifier     1.0   0.488456


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Peso populacional

In [50]:
results = []

for name, model in models_com_peso.items():
    model.fit(X_train, y_train)
    results.append({  'Model'    : name
                    , 'Treino'   : balanced_accuracy_score(y_train, model.predict(X_train))
                    , 'Validação': balanced_accuracy_score(y_valid, model.predict(X_valid))})

# Exibir resultados em um DataFrame
results_df_pesos = pd.DataFrame(results)
print(results_df_pesos.sort_values(by='Validação', ascending=False))

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [23:01:59] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "class_weight" } are not used.

  warnings.warn(smsg, UserWarning)


                                           Model    Treino  Validação
4                    Stochastic Gradient Descent  0.626371   0.604998
5                               Ridge Classifier  0.633253   0.604869
2  SVC(kernel="linear",C=0.025,probability=True)  0.628776   0.588861
1                              SVC(gamma=2, C=1)  0.877194   0.551886
0                                  Random Forest  0.875458   0.513532
3                                        XGBoost  0.581602   0.511116


In [57]:
results_df_smote.set_index('Model')

,Treino,Validação
Model,,
Random Forest,0.917167,0.556878
"SVC(gamma=2, C=1)",0.893956,0.539013
"SVC(kernel=""linear"",C=0.025,probability=True)",0.652402,0.602743
K-Nearest Neighbors,0.770521,0.555695
Radius Neighbors,0.500000,0.500000
Gradient Boosting,0.741929,0.597491
XGBoost,0.822885,0.525878
CatBoost,0.837963,0.554377
Neural Network,0.656031,0.608982


In [64]:
      results_df_pesos.set_index('Model')\
.join(results_df_smote.set_index('Model'), how = 'outer', lsuffix='_pesos', rsuffix='_smote')\
.join(results_df_under.set_index('Model'), how = 'outer',                   rsuffix='_under')\
.join(results_df_over .set_index('Model'), how = 'outer',                   rsuffix='_over').sort_values(by='Validação_smote', ascending=False)

,Treino_pesos,Validação_pesos,Treino_smote,Validação_smote,Treino,Validação,Treino_over,Validação_over
Model,,,,,,,,
MultinomialNB,NaN,NaN,0.650713,0.613733,1.0,0.503078,1.0,0.503078
Neural Network,NaN,NaN,0.656031,0.608982,1.0,0.488456,1.0,0.498214
Ridge Classifier,0.633253,0.604869,0.662600,0.607360,1.0,0.488456,1.0,0.488456
"SVC(kernel=""linear"",C=0.025,probability=True)",0.628776,0.588861,0.652402,0.602743,1.0,0.488456,1.0,0.488456
Stochastic Gradient Descent,0.626371,0.604998,0.668731,0.600743,1.0,0.488456,1.0,0.498214
AdaBoostClassifier,NaN,NaN,0.681494,0.598383,1.0,0.500250,1.0,0.504489
Gradient Boosting,NaN,NaN,0.741929,0.597491,1.0,0.499501,1.0,0.500430
Random Forest,0.875458,0.513532,0.917167,0.556878,1.0,0.500626,1.0,0.500626
K-Nearest Neighbors,NaN,NaN,0.770521,0.555695,1.0,0.498214,1.0,0.488456


In [51]:
results_df_pesos.join(results_df_smote)

ValueError: columns overlap but no suffix specified: Index(['Model', 'Treino', 'Validação'], dtype='object')